In [ ]:
# Make an object which can be initiallized (to load all the nessecary models etc so it doesnt
# have to be done each time). Make individual class detector methods.
# Make a predict class pipeline that goes through the flowchart,
# initiating each class detector when needed.

In [579]:
import math
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
from PIL import Image
import tensorflow as tf
from pathlib import Path
import torch

class Explanation_error_detector:
    
    def __init__(self,perplexity_threshold=137, model_pickles = ['Information_error_model.tf','AE_Mismatch_model.tf','answer_ohe']):
        ### Text Encoder:
        # Load Language Model
        # Load pre-trained model (weights)
        self.model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
        self.model.eval()
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
        
        ### Img Encoder:
        # Not used for the moment to save time
#         self.image_features_extract_model = tf.keras.applications.Xception(
#             include_top=True,
#             weights="imagenet",
#             input_tensor=None,
#             input_shape=None,
#             pooling=None,
#             classes=1000,
#         )
        
        ### Detection models:
        self.Inf_Err_model = tf.keras.models.load_model(model_pickles[0])
        self.AE_model = tf.keras.models.load_model(model_pickles[1])
        
        ### One Hot Encoder for answers:
        self.one_hot = pickle.load(open(model_pickles[2], 'rb'))
        
        ### For clarity error:
        self.perplexity_threshold = perplexity_threshold
        
    def perplexity_score(self,sentence):
        if type(sentence) != str:
            sentence = sentence.to_string()
        tokenize_input = self.tokenizer.tokenize(sentence)
        tensor_input = torch.tensor([self.tokenizer.convert_tokens_to_ids(tokenize_input)])
        loss=self.model(tensor_input, lm_labels=tensor_input)
        return math.exp(loss)
    
    # Doesn't work yet: assume features have been pre-generated for now.
#     def encode_image(self,img_path):
#             img = tf.io.read_file(img_path)
#             img = tf.image.decode_jpeg(img, channels=3)
#             img_pre = tf.image.resize(img, (299, 299))
#             img = tf.keras.applications.xception.preprocess_input(img_pre)
#             img = tf.reshape(img,(1,299,299,3),output)
#             print(img)
#             img = self.image_features_extract_model(img).numpy()
#             return img
    
    def detect_clarity_error(self,explanation):
       # Kan je vgm ook een probability van maken op een bepaalde manier maar eh
        if self.perplexity_score(explanation)>self.perplexity_threshold:
            return True
        else:
            return False
    def detect_information_error(self, explanation,image, npy=True):
        
        print(explanation.shape, image.shape)
        # If npy=True --> Explanation and image should be numpy arrays containing the encodings
        pred = self.Inf_Err_model.predict([np.vstack(image),np.vstack(explanation)])
        return pred
        # Transform image into encodings (skipped) -> predict using IE model -> Return probab (only return label in full model)
        
    def detect_AE_error(self, explanation, qstr, ans, npy=True): # Note: ans is pred from dataset
        # If npy=True --> explanation and question should be numpy arrays containing the encodings
        pred = self.AE_model.predict([np.vstack(qstr), np.vstack(explanation)])
        pred_ans = self.one_hot.inverse_transform(pred)
        
        if type(ans) != str:
            ans = ans.to_string()

            if (pred_ans[0][0] == ans):
                return 0
            else:
                return 1
        else:
            if (pred_ans[0][0] == ans):
                return 0
            else:
                return 1
        
    def detect_error(self,explanation, exp_enc, image, qstr, pred, ans, npy= True):

        if type(pred) == str:
            answer_match = pred == ans
        else:
            answer_match = (pred == ans).bool()
        if self.detect_clarity_error(explanation) or exp_enc.shape[2] > 22:
            return 1
        elif self.detect_information_error(exp_enc, image)>0.5:
            return 2
        elif self.detect_AE_error(exp_enc, qstr, pred):
            return 4
        elif answer_match: # answer correct
            return 5
        else:
            return 3 # reasoning error
    
    def pad_text_npy(self,text):
        s = text.shape
        if s[1] >22:
            np.resize(text,(s[0],22,s[2]))
            out = text
        else:
            out = np.pad(text, ((0,0),(0,22-s[1]),(0,0)),'constant', constant_values = 0)

        return out
        

In [580]:
eer = Explanation_error_detector()
#print(eer.encode_image(str(Path('D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000532732.jpg'))))

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [581]:
eer.Inf_Err_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 22, 768)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 16896)        0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 17896)        0           input_1[0][0]                    
                                                                 flatten[0][0]                

In [582]:
ploenkie = data.sample()

print("INF ONLY:", eer.detect_information_error(ploenkie.exp_enc, ploenkie.img_enc))
ploenkie_pred = ploenkie.apply(lambda x:
                               eer.detect_error(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                                                np.array([x.qstr_enc]), x.pred, x.ans), axis=1)


eer.detect_error(ploenkie.exp, ploenkie.exp_enc, ploenkie.img_enc, ploenkie.qstr_enc, ploenkie.pred, ploenkie.ans)
# pred_label = 5 if (eer.one_hot.inverse_transform(ploenkie_pred)[0][0] == ploenkie.ans).bool() else 4
# pred_label, ploenkie.label

print('pred:',ploenkie_pred, 'true:',ploenkie.label)

#eer.detect_clarity_error(ploenkie.exp)


(1,) (1,)
INF ONLY: [[0.56771785]]
pred: 44800003    1
dtype: int64 true: 44800003    5
Name: label, dtype: int64


In [583]:
data

#DIT NOG EEN KEER DOEN MET GOEIE TEST/TRAIN VERDELING
data['pred_label_full'] = data.apply(lambda x:
                               eer.detect_error(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                                                np.array([x.qstr_enc]), x.pred, x.ans), axis=1)

(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 7

(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 7

(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 7

(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 768) (1, 1000)
(1, 1, 22, 7

In [589]:
yhat = data.pred_label_full
y = data.label

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y,yhat))
print(confusion_matrix(y,yhat))
print(accuracy_score(y,yhat))

              precision    recall  f1-score   support

           1       0.17      0.68      0.28       190
           2       0.70      0.56      0.63       816
           3       0.14      0.11      0.13        35
           4       0.18      0.12      0.14        92
           5       0.87      0.50      0.64       826
           6       0.00      0.00      0.00         9

    accuracy                           0.52      1968
   macro avg       0.34      0.33      0.30      1968
weighted avg       0.68      0.52      0.56      1968

[[129  46   0   5  10   0]
 [267 461  21  20  47   0]
 [ 17  13   4   1   0   0]
 [ 52  23   0  11   6   0]
 [271 113   3  24 415   0]
 [  5   3   0   0   1   0]]
0.5182926829268293


c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [207]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
data = pd.read_pickle('Post_Processing_with_Encoding_ans_pred.pkl')
ohe = OneHotEncoder().fit(data.pred.to_numpy().reshape(-1,1)) # Deze pickelen?
import pickle
with open("answer_ohe", "wb") as f: 
    pickle.dump(ohe, f)

ValueError: Expected 2D array, got 1D array instead:
array=['cake' 'couch' 'yes' ... 'fire hydrant' 'no' 'yes'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [198]:
for enc,exp in zip(data.exp_enc, data.exp):
    if not enc.shape[1] == len(exp.split()):
        print(exp, enc.shape[1], len(exp.split()) )

it is cut in a slices and full of toppings 11 10
there are trains and a common hydrant in the background 11 10
the sign says dixie hwy and thats in the united states 13 11
he is on a surfboard 6 5
there is a stove and refrigarator and a sink in it 13 11
it has wings and is perched on a tree brance 11 10
he is on a snowboard on the snow 9 8
the sky is blue and there isnt a cloud in the sky 13 12
he is on a snowboard on the snow 9 8
he is on a snowboard on the snow 9 8
they are wearing uniforms and tuxedos 7 6
the colors and the racquets are dull 8 7
his hair is brushing his shirt colar 8 7
it is a macbook 5 4
it has a retractable handle 6 5
he is wearing a helmet and is snowboarding 9 8
he is on a surfboard 6 5
they are on a snowboard in the snow 9 8
the giraffe is in a pen 7 6
they are on skateboards 5 4
he is on a snowboard on the snow 9 8
they are on a tennis court with rackets and tennis balls 12 11
it is a round round piece of marbled meat in a dish 13 12
he is on a snowboard on th

In [199]:
m = 0 #Find size of biggest explanation to potentially use in model as cutoff point. (all longer explanations are probably clarity errors.) 
for x in data.exp[data.label == 5]:
    l = len(x.split())
    if l>m:
        m = l
m

17

In [213]:
data.exp_enc = data.exp_enc.apply(eer.pad_text_npy)
data.qstr_enc = data.qstr_enc.apply(eer.pad_text_npy)

# for col in ('exp_enc','qstr_enc')
# data.exp_enc = data.exp_enc.apply(pad_exp)
data.exp_enc.iloc[3].shape


(1, 22, 768)

In [201]:
foo = np.zeros((1,20,50))
foo.resize((1,15,30))
foo.shape

(1, 15, 30)

In [203]:
t = eer.one_hot.transform(np.array('yes').reshape(-1,1)).todense()

eer.one_hot.inverse_transform(t)

array([['yes']], dtype=object)